# Importing Libraries
***

In [1]:
from neo4j import GraphDatabase
from pandas import DataFrame
import pandas as pd
import numpy as np
import re
import csv
import ast
import json

# Connecting to Neo4j
***

In [2]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "123123123"))

# Deffining Useful Functions
***

In [3]:
def run_query(query): 
     with driver.session() as session:
        result = session.run(query)
        #print(result.single()[0])
        
        
def delete_all_nodes():
    query = '''
    MATCH(n) DETACH DELETE(n)
    '''
    run_query(query)

def record_count():
    with driver.session() as session:
        result = session.run("MATCH (n) RETURN count(n) LIMIT 100")
        print(result.single())

def delete_nodes(var):
    query = f'''
    MATCH (c:{var})
    DELETE c

    '''
    run_query(query)

# Recommender
## Find/Define Research Communities
***


In [4]:
def get_community_keywords(query): 
     with driver.session() as session:
        result = session.run(query)
        keywords= []
        for record in result:
            keyword = record['n.name']
            keywords.append(keyword)
        return keywords

query = '''MATCH (n:Keyword) 
WHERE n.name IN ['data modeling', 'big data', 'data', 'data storage', 'processing', 'data querying', 'data management', 'indexing']
RETURN n.name
'''

result = get_community_keywords(query)
#result.head(10)
print(result)

['big data', 'indexing', 'data management', 'data storage', 'data']


In [5]:
query = '''
MERGE (c: Community {name: 'database'})
'''
run_query(query)

In [6]:
for elem in result:
    with driver.session() as session:
        result = session.run(
        '''MATCH (c:Community{name:'database'}), (k:Keyword {name:$keyword}) MERGE (c)-[:has_keyword]->(k)''', keyword = elem)

## Conferences and journals related to the database community
### A conference or a journal is related to the database community if 90% of the papers published in a conference/journal contain one of the keywords of the database
***

In [7]:
def get_related_conferences(query): 
     with driver.session() as session:
        result = session.run(query)
        commun_conf_dict = []
        for record in result:
            com_id = record['ID(c)']
            conf_id = record['ID(conf)']
            commun_conf_dict.append((com_id,conf_id))
        return commun_conf_dict

def get_related_journals(query): 
     with driver.session() as session:
        result = session.run(query)
        journal_conf_dict = []
        for record in result:
            com_id = record['ID(c)']
            journ_id = record['ID(j)']
            journal_conf_dict.append((com_id,journ_id))
        return journal_conf_dict

query_conf = '''
MATCH (c:Community)-[:has_keyword]->(k:Keyword)
WHERE c.name = 'database'
WITH c, k
MATCH (p:Paper)-[:published_in_conference]->(conf:Conference)
WHERE (p)-[:has_keyword]->(k)
WITH c, conf, k, count(DISTINCT p) as num_papers
WHERE num_papers / toFloat(size((conf)<-[:published_in_conference]-())) >= 0.9
RETURN ID(c), ID(conf)
'''
query_journal = '''
MATCH (c:Community)-[:has_keyword]->(k:Keyword)
WHERE c.name = 'database'
WITH c, k
MATCH (p2:Paper)-[:published_in_journal]->(j:Journal)
WHERE (p2)-[:has_keyword]->(k)
WITH c, j, k, count(DISTINCT p2) as num_journal_papers
WHERE num_journal_papers / toFloat(size((j)<-[:published_in_journal]-())) >= 0.9
RETURN ID(c), ID(j)
'''
confs = get_related_conferences(query_conf)
journs = get_related_journals(query_journal)

print(confs)
print(journs)



[(0, 119334), (0, 119714), (0, 118848), (0, 119048), (0, 119392), (0, 119758)]
[(0, 120970), (0, 120326)]


### Extending the graph by adding relations between the communities and the related conferences/journals

In [8]:
for elem in confs:
    with driver.session() as session:
        result = session.run(
        '''MATCH (n1), (n2)
        WHERE ID(n1) = $id1 AND ID(n2) = $id2
        MERGE (n1)-[:is_related_to]->(n2)''', id1 = elem[1], id2 =elem[0])

In [9]:
for elem in journs:
    with driver.session() as session:
        result = session.run(
        '''MATCH (n1), (n2)
        WHERE ID(n1) = $id1 AND ID(n2) = $id2
        MERGE (n1)-[:is_related_to]->(n2)''', id1 = elem[1], id2 =elem[0])

## Identifying the top papers of the related conferences/journals of database community

### Create graph projection of database community papers and citations

In [4]:
query_prev = '''CALL gds.graph.project.cypher('citedByDatabaseComm', 'MATCH (c:Community)
WHERE c.name = "database"
WITH c
MATCH (c)<-[:is_related_to]-(j :Journal) <- [:published_in_journal] - (p1:Paper)
WITH c,j,p1
MATCH (c)<-[:is_related_to]-(conf :Conference) <- [:published_in_conference] - (p:Paper)
WITH c,conf,j,p,p1
WITH Collect(p1) + Collect(p) AS papers, c,j,conf
UNWIND papers as p3
MATCH a = (f:Paper)-[:cites]->(p3:Paper)
WITH COLLECT(f)+COLLECT(p3) as nodes
UNWIND nodes as nodes_item
RETURN id(nodes_item) as id', 'MATCH (c:Community)
WHERE c.name = "database"
WITH c
MATCH (c)<-[:is_related_to]-(j :Journal) <- [:published_in_journal] - (p1:Paper)
WITH c,j,p1
MATCH (c)<-[:is_related_to]-(conf :Conference) <- [:published_in_conference] - (p:Paper)
WITH c,conf,j,p,p1
WITH Collect(p1) + Collect(p) AS papers, c,j,conf
UNWIND papers as p3
MATCH a = (f:Paper)-[:cites]->(p3:Paper)
RETURN id(f) as source, id(p3) as target, "cites" as type')'''

run_query(query_prev)

### Setting up the queries for PageRank Algorithm

In [5]:
query1 = '''MATCH (c:Community)
WHERE c.name = "database"
WITH c
MATCH (c)<-[:is_related_to]-(j :Journal)
WITH c,j
MATCH (c)<-[:is_related_to]-(conf :Conference)
WITH c,j,conf
MATCH (p)-[:published_in_journal]->(j)
WITH p,c,j,conf
MATCH (p1)-[:published_in_conference]->(conf)
WITH Collect(p1) + Collect(p) AS papers
WITH apoc.convert.toList(papers) AS regularList

CALL gds.pageRank.stream('citedByDatabaseComm')
YIELD nodeId, score
RETURN nodeId AS id_paper, score
ORDER BY score DESC, id_paper ASC
'''

query2 = '''MATCH (c:Community)
WHERE c.name = "database"
WITH c
MATCH (c)<-[:is_related_to]-(j :Journal)
WITH c,j
MATCH (c)<-[:is_related_to]-(conf :Conference)
WITH c,j,conf
MATCH (p)-[:published_in_journal]->(j)
WITH p,c,j,conf
MATCH (p1)-[:published_in_conference]->(conf)
WITH Collect(p1) + Collect(p) AS papers
UNWIND papers as lista
RETURN DISTINCT id(lista) as ids_comm
'''

### Setting up the queries for PageRank Algorithm

In [6]:
def page_rank(query): 
     with driver.session() as session:
        result = session.run(query)
        records = []
        for record in result:
            id_paper = record['id_paper']
            score = record['score']
            records.append({'Paper_ID': id_paper, 'page_rank_score': score})

        df = pd.DataFrame.from_records(records)
        return df
    
def dataset_community_papers(query): 
     with driver.session() as session:
        result = session.run(query)
        records = []
        for record in result:
            ids_comm = record['ids_comm']
            records.append({'ids_comm': ids_comm})

        df = pd.DataFrame.from_records(records)
        return df

df_pagerank = page_rank(query1)
df_community_database = dataset_community_papers(query2)

id_list = list(df_community_database['ids_comm'])
filtered_page_ranks = df_pagerank[df_pagerank['Paper_ID'].isin(id_list)]
filtered_page_ranks

,Paper_ID,page_rank_score
0,23622,2.1900
1,17232,1.4250
2,20411,1.4250
3,18459,1.2975
4,18627,1.2975
5,15819,0.9150
6,16219,0.9150
7,17028,0.6600


### Extending the graph by adding a relationship between the 100 top papers of a community

In [7]:
for i in list(filtered_page_ranks[:100]['Paper_ID']):
    with driver.session() as session:
        result = session.run('''MATCH (c:Community), (p:Paper)
        WHERE c.name = 'database' AND ID(p) = $id_paper
        MERGE (p)-[:is_top_100]->(c)''', id_paper = i)

### Identifying potential reviewers of the community

In [4]:
# potential good match to review database papers
def good_match_reviewer(query): 
     with driver.session() as session:
        result = session.run(query)
        records = []
        for record in result:
            id_author = record['a.id']
            author_name = record['a.name']
            com_name = record['c.name']
            records.append({'author_id': id_author, 'author_name': author_name, 'community_name': com_name})

        df = pd.DataFrame.from_records(records)
        return df



query_pot_reviewers = '''
MATCH (a:Author) -[:wrote]->(p:Paper)-[:is_top_100]->(c:Community) RETURN a.id, a.name ,c.name
'''
x = good_match_reviewer(query_pot_reviewers)
x

,author_id,author_name,community_name
0,53f43600dabfaedce552bb22,Michael Stonebraker,database
1,53f430b0dabfaeb1a7bb524d,Richard R. Berman,database
2,540544efdabfae8faa5bb0ee,Stéphane Marchand-Maillet,database
3,53f78543dabfae92b40c2294,Hisham Mohamed,database
4,53f4d1dfdabfaef0f0f8093d,Yin Liao,database
5,5429fcc8dabfae5848a84a62,Guoliang Chen,database
6,56cb18b4c35f4f3c6565c9b8,Yunquan Zhang,database
7,53f49d2cdabfaee1c0baddc7,Tao Luo,database
8,53f59ae6dabfaeec23f8045b,Joel H Saltz,database
9,53fa0780dabfae7f97b00f3e,Fusheng Wang,database


### Identifying gurus - authors of, at least, two papers among the top-100 identified

In [11]:
# potential good match to review database papers
def get_gurus(query): 
     with driver.session() as session:
        result = session.run(query)
        records = []
        for record in result:
            id_author = record['a'].element_id
            id_com = record['c'].element_id
            number_of_papers_out_of_top_100 = record['num_top_100_papers']
            records.append({'author_id': id_author, 'community_id': id_com, 'out_of_100': number_of_papers_out_of_top_100})

        df = pd.DataFrame.from_records(records)
        return df

query_gurus = '''MATCH (a:Author)-[r:wrote]->(p:Paper)-[:is_top_100]->(c:Community)
WITH a, c, count(p) as num_top_100_papers
WHERE num_top_100_papers >= 2
RETURN a,c, num_top_100_papers
'''

gurus = get_gurus(query_gurus)
gurus

,author_id,community_id,out_of_100
0,31091,0,2


### Extending the graph by adding a relationship between Gurus and the specific Community

In [14]:
for i,row in gurus.iterrows():
    with driver.session() as session:
        result = session.run('''MATCH (a:Author), (c:Community)
                            WHERE ID(a) = $id_author AND ID(c) = $id_com
                            MERGE (a)-[:is_guru_for]->(c)''', id_author = int(row['author_id']), id_com = int(row['community_id']))